In [24]:
import pandas as pd
import mlxtend as ml
import numpy as np
import sklearn as sk
import sqlite3
import re
import math

In [25]:
ASSETS_DIR = "./assets"

WAREHOUSE_FILE = f"{ASSETS_DIR}/data_warehouse_2.csv"

In [26]:
main_df = pd.read_csv(WAREHOUSE_FILE)

main_df

,municipios,quantidade_ucs,area_coberta,populacao_2010,populacao_2011,populacao_2012,populacao_2013,populacao_2014,populacao_2015,populacao_2016,...,empregados_2012,empregados_2013,empregados_2014,empregados_2015,empregados_2016,empregados_2017,empregados_2018,empregados_2019,empregados_2020,estabelecimentos
0,ABDON BATISTA,0,0,2644,2644,2635,2656,2643,2630,2617,...,104,246,581,1111,292,232,352,276,268,181
1,ABELARDO LUZ,0,0,17151,17151,17200,17584,17651,17717,17782,...,3208,3291,3484,3494,3348,3378,3303,3505,3670,1693
2,AGROLÂNDIA,0,0,9440,9440,9552,9957,10115,10272,10427,...,2639,2944,3083,2776,2557,2457,2441,2622,2700,1571
3,AGRONÔMICA,0,0,4954,4954,4985,5172,5239,5306,5371,...,699,718,769,761,746,732,796,849,887,655
4,ÁGUA DOCE,2,109,6971,6971,6979,7110,7121,7132,7143,...,1099,1144,1227,1270,1313,1273,1332,1356,1452,1026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,WITMARSUM,0,0,3627,3627,3653,3769,3805,3841,3876,...,818,873,1022,1005,988,1014,1040,1054,1063,297
290,XANXERÊ,0,0,44643,44643,45140,46981,47679,48370,49057,...,11218,11025,11636,11804,12045,11826,11703,12004,11884,769
291,XAVANTINA,0,0,4122,4122,4103,4124,4095,4067,4039,...,275,300,327,339,317,305,278,314,189,305
292,XAXIM,0,0,25933,25933,26145,27039,27336,27630,27921,...,6623,7295,8327,8363,8531,8608,9129,9381,10109,916


In [30]:
# main_df[["municipios", "area_coberta"]]
total_area = main_df["area_coberta"].sum()
main_df["area_coberta %"] = main_df["area_coberta"] / total_area * 100.0

year_employenment_rate = (main_df["empregados_2020"] - main_df["empregados_2010"]) / (20 - 10)

year_population_rate = (main_df["populacao_2020"] - main_df["populacao_2010"]) / (20 - 10)

main_df["taxa_de_empregados (2010-2020)"] = year_employenment_rate
main_df["taxa_de_populacao (2010-2020)"] = year_population_rate

main_df.sort_values(["taxa_de_populacao (2010-2020)", "taxa_de_empregados (2010-2020)", "area_coberta"], ascending=False)

,municipios,quantidade_ucs,area_coberta,populacao_2010,populacao_2011,populacao_2012,populacao_2013,populacao_2014,populacao_2015,populacao_2016,...,empregados_2015,empregados_2016,empregados_2017,empregados_2018,empregados_2019,empregados_2020,estabelecimentos,area_coberta %,taxa_de_empregados (2010-2020),taxa_de_populacao (2010-2020)
88,FLORIANÓPOLIS,20,279607,427298,427298,433158,453285,461524,469690,477798,...,149780,146759,142896,147455,154647,147935,148,9.100383,2046.8,8152.8
137,JOINVILLE,7,46460,520905,520905,526338,546981,554601,562151,569645,...,174166,167699,173546,188388,188096,188843,20668,1.512136,2772.4,7675.3
38,BLUMENAU,3,62296,312635,312635,316139,329082,334002,338876,343715,...,112772,108082,109408,110802,109849,109955,374,2.027551,-133.2,4922.0
67,CHAPECÓ,1,1604,186337,186337,189052,198188,202009,205795,209553,...,66600,65433,67395,68490,74492,78878,1158,0.052205,1978.6,3767.6
248,SÃO JOSÉ,1,26,212587,212587,215278,224779,228561,232309,236029,...,95212,90642,91762,88243,93792,102221,563,0.000846,2962.1,3759.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,IMARUÍ,1,83908,11540,11540,11411,11301,11117,10933,10752,...,648,692,639,617,646,689,493,2.730958,24.2,-159.2
122,IRATI,0,0,60357,62808,2067,2067,2046,2025,2004,...,157,187,198,205,226,245,2078,0.000000,11.0,-165.8
73,CORREIA PINTO,0,0,14613,14613,14447,14301,14063,13826,13591,...,2317,2318,2366,2420,2538,2543,176,0.000000,40.7,-206.0
109,IÇARA,1,154859,59616,59616,49238,51416,52284,53145,53998,...,14194,13912,14179,14981,15898,17147,11167,5.040204,485.5,-236.9
